In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\NLP\\SentimentAnalysis\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'd:\\Data Science\\NLP\\SentimentAnalysis'

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelPredictionConfig:
    root_dir: Path
    tokens_path: Path
    final_model_path: Path
    save_model_performance: str
   


In [40]:
from SentimentAnalysis.constants import *
from SentimentAnalysis.utils.common import read_yaml
from SentimentAnalysis.utils.common import create_directories
import joblib

In [41]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.dataStore_root])

    #2. To be updated configuration inside config into Configuration

    def get_model_prediction_config(self) -> ModelPredictionConfig:
          config = self.config.model_prediction

          create_directories([config.root_dir])

          model_prediction_config = ModelPredictionConfig(
                root_dir=config.root_dir,
                tokens_path=config.tokens_path, 
                final_model_path=config.final_model_path,
                save_model_performance=config.save_model_performance
                )

          return model_prediction_config
    

In [42]:
import os
import pandas as pd
import numpy as np
from SentimentAnalysis.logging import logger
import tensorflow as tf
import joblib
from nltk.tokenize.toktok import ToktokTokenizer




In [53]:
from SentimentAnalysis.config.configuration import ConfigurationManager
from SentimentAnalysis.components.eda_data_cleaning import DataCleaner
import tensorflow as tf
import joblib
from SentimentAnalysis.logging import logger
from pathlib import Path


class PredictionPipeline:
    def __init__(self):
        self.config = ConfigurationManager().get_model_prediction_config()
        self.data_cleaner = DataCleaner(self.config)
        self.max_review_length = 300
        #self.tokenizer = tf.keras.preprocessing.text.Tokenizer()
        #self.token = joblib.load(self.config.tokenizer_path)
        self.model = joblib.load(self.config.final_model_path)

    def predict(self):
        token = joblib.load('dataStore/data_transformation/tokenizer.joblib')
        #token = joblib.load(self.config.tokens_path)
        text = input("Enter a review: ")
        # Clean the user input using DataCleaner methods
        text = self.data_cleaner.denoise_text(text)  # Call denoise_text on the instance
        text = self.data_cleaner.remove_stopwords(text)  # Call remove_stopwords on the instance
        text = self.data_cleaner.apply_stemming(text)  # Call apply_stemming on the instance
        print("Cleaned input:", text)

        # token = joblib.load('dataStore/data_transformation/tokenizer.joblib')
        
        # Tokenize and pad the input
        user_input_sequence = token.texts_to_sequences([text])
        print("User input sequence:", user_input_sequence)

        user_input_padded = tf.keras.preprocessing.sequence.pad_sequences(user_input_sequence, self.max_review_length, padding='pre')
        print("User input padded:", user_input_padded)

        # Make prediction
        prediction_prob = self.model.predict(user_input_padded)[0][0]
        
        # Display the result
        if prediction_prob > 0.5:
            print("predicted_sentiment: Positive")
        else:
            print("predicted_sentiment: Negative")

        print("Probability:", prediction_prob)


In [56]:
pipeline = PredictionPipeline()

pipeline.predict()

[2023-08-12 15:58:28,888: INFO: common: YAML file loaded successfully: config\config.yaml]
[2023-08-12 15:58:28,890: INFO: common: YAML file loaded successfully: params.yaml]
[2023-08-12 15:58:28,895: INFO: common: YAML file loaded successfully: schema.yaml]
[2023-08-12 15:58:28,895: INFO: common: Created directory at: dataStore]
[2023-08-12 15:58:28,899: INFO: common: Created directory at: dataStore/model_prediction]
Cleaned input: im go disagre previous comment side maltin one second rate excess vicious western creak groan tri put across central theme wild west tame kick asid steadi march time would like tradit butch cassidi sundanc kid lack film poignanc charm andrew mclaglen direct limp final 30 minut real botch incomprehens strategi part hero charlton heston chris mitchum someon give holler explain set hillsid fire someth callous whole treatment rape scene woman reaction afterward certain ring true coburn plenti nasti half breed escap convict reveng fellow escape underdevelop they